Richard,

Thanks for the article. Here are some problems I have with that interpretation of Shcrodinger's cat 

In [12]:
from __future__ import division

In [13]:
import random

In [17]:
#Give me a random percentage between 0 and 100
a = random.randint(1,100)

Let's assume we have the typical Schrödinger's cat experiment. We'll let the experiment run for a amount of time such that the probability of the cat being either alive or dead is exactly 50%. Then, we'll assume the vial of poison is destroyed, thus at all times after the experiment, the chance that the cat remains at 50%, and we have the job of determining when the waveform collapses. To save on cats, we'll instead use the function experiment() to give a result of either 1, for alive, or 0, for dead. experiment() will generate these result randomly with a probability of 50% for each result.

In [1]:
#This function simulates the Schrödinger's cat experiment by outputting either a 1, for alive, or a 0.
def experiment():
    alive = random.randint(0,1) # Randomly decide between a 0 and a 1
    return alive

Now let's set up the rest of the experiment. Attached to the experimental device is a printer. The experimental device learns the results of the experiment, but instead of just printing the result accurately, it'll print out a result that might not be true. For the first round, we'll print out either "The cat is alive" or "The cat is dead" with a 50% probability. Thus, there is actually no information about the experiment contained in the first printout.

In [42]:
all_results=[] # we'll use this to collect all of our results in
def print_result(result, veracity):
    if result==1 and  veracity == True or result==0 and veracity == False:
        print("The cat is alive")
        all_results.append(1)
    else:
        print("The cat is dead")
        all_results.append(0)

OK, now we'll got our experiment and our printer set up, let's run the experiment!

In [14]:
#Run the experiment and get the result
result = experiment()

In [38]:
probability_of_truth = 50
print(probability_of_truth)
random_number = random.randint(1,100) # Generate a random number
if random_number<probability_of_truth:
    print_result(result, True)
else:
    print_result(result, False)

50
The cat is dead


Now the experiment has been conducted and a result has been printed, although the result is meaningless. Let's do that a few more times, except with each time we'll increase the probability that a True result is printed by one percentage point. Thus the next printout will be the truth with a probability of 51% and a lie with a probability of 49%. And the one after that will have a 52% chance of being true, and so on until 60%. 

In [30]:

trials = 9
for x in range(trials):
    probability_of_truth = 51 + x # Starting at 51% and then increasing it by 1% every iteration
    #print("\nWith a probability of {probability}%:".format(probability=probability_of_truth))
    random_number = random.randint(1,100) # Generate a new random number each time
    if random_number<probability_of_truth:
        print_result(result, True)
    else:
        print_result(result, False)
#print (count / trials)

The cat is alive
The cat is dead
The cat is alive
The cat is dead
The cat is dead
The cat is alive
The cat is alive
The cat is alive
The cat is alive


If it so happens that the person receiving the printout, we'll call her Alice, is a statistician, she could quickly calculate the true probability:

In [41]:
all_results

[]

In [40]:
trials = 50
count = 0

for x in range(trials):
    prob_of_truth = 50 + 100*(x/(2*trials))
    print(prob_of_truth)
    a = random.randint(1,100)
    if a<prob_of_truth:
        truth(result)
    else:
        lie(result)
print (count / trials)

50.0
The cat is dead
51.0
The cat is dead
52.0
The cat is dead
53.0
The cat is alive
54.0
The cat is dead
55.0
The cat is dead
56.0
The cat is dead
57.0
The cat is alive
58.0
The cat is alive
59.0
The cat is dead
60.0
The cat is alive
61.0
The cat is dead
62.0
The cat is dead
63.0
The cat is alive
64.0
The cat is alive
65.0
The cat is dead
66.0
The cat is dead
67.0
The cat is dead
68.0
The cat is alive
69.0
The cat is dead
70.0
The cat is alive
71.0
The cat is dead
72.0
The cat is alive
73.0
The cat is alive
74.0
The cat is alive
75.0
The cat is alive
76.0
The cat is dead
77.0
The cat is alive
78.0
The cat is alive
79.0
The cat is alive
80.0
The cat is alive
81.0
The cat is dead
82.0
The cat is alive
83.0
The cat is alive
84.0
The cat is alive
85.0
The cat is alive
86.0
The cat is alive
87.0
The cat is alive
88.0
The cat is alive
89.0
The cat is alive
90.0
The cat is alive
91.0
The cat is dead
92.0
The cat is alive
93.0
The cat is alive
94.0
The cat is alive
95.0
The cat is alive
96.0


In [23]:
def truth(result):
    if result==1:
        print("The cat is alive")
    else:
        print("The cat is dead")
def lie(result):
    if result==1:
        print("The cat is dead")
    else:
        print("The cat is alive")

In [30]:
for x in range(5):
    print(experiment())

0
0
1
0
1


In [32]:
result = experiment()

In [ ]:
#The other person learns whether the truth or a lie was told

In [22]:
#Two people could also lie to each other with a probability

In [9]:
print_result(0, 1)

The cat is dead
